In [ ]:
!pip install pymc==5.10.4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datetime

In [ ]:
# update with the appropriate training data path
df_winter = pd.read_csv('',index_col=0, parse_dates=True)

df_winter.head()

In [ ]:
df_winter.isnull().sum()

In [ ]:
df_winter = df_winter.dropna()

In [ ]:
df_winter.isnull().sum()

In [ ]:
df_winter['fog_index_5d'].eq(0).sum()

In [ ]:
df_winter.shape

In [ ]:
df_filtered = df_winter[df_winter['fog_index_5d'] != 0]

In [ ]:
df_winter = df_filtered

In [ ]:
df_winter.shape

In [ ]:
df_winter['fog_index_5d'].eq(0).sum()

In [ ]:
df_winter = df_winter[df_winter['energy_loss'] <= 1500000]

In [ ]:
df_winter.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X = df_winter.drop(["fog_index_6h","fog_index_1d","fog_index_3d","fog_index_5d"], axis=1)  # Features
y = df_winter["fog_index_5d"]

In [ ]:
scaler = StandardScaler()
scaler.fit(X)  # Fit scaler on training data

X_train_scaled = scaler.transform(X)

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)

In [ ]:
X_train_scaled_df.columns

In [ ]:
y_train = y

In [ ]:
import pymc as pm

with pm.Model() as beta_regression_model:
    # Priors
    beta_0 = pm.Normal('beta_0', mu=0, sigma=100)
    beta_1 = pm.Normal('beta_1', mu=0, sigma=100)
    beta_2 = pm.Normal('beta_2', mu=0, sigma=100)
    beta_3 = pm.Normal('beta_3', mu=0, sigma=100)
    beta_4 = pm.Normal('beta_4', mu=0, sigma=100)
    beta_5 = pm.Normal('beta_5', mu=0, sigma=100)
    beta_6 = pm.Normal('beta_6', mu=0, sigma=100)
    beta_7 = pm.Normal('beta_7', mu=0, sigma=100)
    beta_8 = pm.Normal('beta_8', mu=0, sigma=100)

    avg_air_temp = pm.MutableData('avg_air_temp', X_train_scaled_df['avg_air_temp'])
    avg_dew_point = pm.MutableData('avg_dew_point', X_train_scaled_df['avg_dew_point'])
    avg_relative_humidity = pm.MutableData('avg_relative_humidity', X_train_scaled_df['avg_relative_humidity'])
    avg_pressure = pm.MutableData('avg_pressure', X_train_scaled_df['avg_pressure'])
    avg_visibility = pm.MutableData('avg_visibility', X_train_scaled_df['avg_visibility'])
    energy_loss = pm.MutableData('energy_loss', X_train_scaled_df['energy_loss'])
    fog_duration = pm.MutableData('fog_duration', X_train_scaled_df['fog_duration'])
    fog_month = pm.MutableData('fog_month', X_train_scaled_df['fog_month'])

    fog_index_5d = pm.MutableData('fog_index', y_train)

    alpha = pm.Uniform('alpha', lower=0, upper=200)
    beta = pm.Uniform('beta', lower=0, upper=200)

    # Precision parameterization
    phi = alpha + beta


    # Mean of the beta distribution
    mu = pm.Deterministic('mu', pm.math.invlogit( beta_0 + beta_1 * avg_air_temp +
                          beta_2 * avg_dew_point +
                          beta_3 * avg_relative_humidity +
                          beta_4 * avg_pressure +
                          beta_5 * avg_visibility +
                          beta_6 * energy_loss +
                          beta_7 * fog_duration +
                          beta_8 * fog_month
                          ))

    # Likelihood
    fog_index_var = pm.Beta('fog_index_var', alpha=mu * phi, beta=(1 - mu) * phi, observed=fog_index_5d)

    # Inference
    trace = pm.sample(draws=2000, tune=2000, chains=2,return_inferencedata=True)

In [ ]:
import arviz as az
az.plot_trace(trace)

In [ ]:
import arviz as az
import matplotlib.pyplot as plt

In [ ]:
with beta_regression_model:  # Reuse the model context
    ppd = pm.sample_posterior_predictive(
        trace,
        var_names=['mu','fog_index_var'],  # Name of output variable
        random_seed=42  # Optional, set a seed for reproducibility
    )
    az.plot_ppc(ppd)
    plt.show()

In [ ]:
trace

In [ ]:
ppd

In [ ]:
def calculate_intervals(hdi,y_test,pp_mean,pp_median):
  test_lower_bounds = []
  test_upper_bounds = []
  test_original_values = []
  test_point_pred_mean = []
  test_point_pred_median =[]
  test_lower_upper_avg = []
  test_dates = []

  for idx in range(len(hdi['fog_index_var_dim_2'])):
    lower_bound = hdi.sel(fog_index_var_dim_2=idx, hdi='lower')['fog_index_var'].values
    upper_bound = hdi.sel(fog_index_var_dim_2=idx, hdi='higher')['fog_index_var'].values
    avg = (upper_bound - lower_bound)/2

    # print(lower_bound['fog_index_var'].values)

    point_pred_mean = pp_mean[idx]
    point_pred_median = pp_median[idx]
    # lb = lower_bound['fog_index_var'].values
    # ub = upper_bound['fog_index_var'].values
    # break
    original = y_test[idx]
    date = y_test.index[idx]
    # print(f"Test Point: {idx}, Range: ({lower_bound:.2f}, {upper_bound:.2f}) ,point mean: {point_pred_mean:.2f},point median: {point_pred_median:.2f}, Original: {original:.2f}, Date: {date}")
    test_lower_bounds.append(lower_bound)
    test_upper_bounds.append(upper_bound)
    test_point_pred_mean.append(point_pred_mean)
    test_point_pred_median.append(point_pred_median)
    test_lower_upper_avg.append(avg)
    test_original_values.append(original)
    test_dates.append(date)

  data = {
    'lower_bound': test_lower_bounds,
    'upper_bound': test_upper_bounds,
    'original': test_original_values,
    'point_pred_mean': test_point_pred_mean,
    'point_pred_median': test_point_pred_median,
    'lower_upper_avg': test_lower_upper_avg,
    'date': test_dates
  }

  return data

In [ ]:
import pandas as pd

def evaluate_range_predictions(data):
    df = pd.DataFrame(data)  # Ensure DataFrame format

    # Coverage: Proportion of original values within bounds
    coverage = (df['original'] >= df['lower_bound']) & (df['original'] <= df['upper_bound'])
    coverage_pct = coverage.mean() * 100

    # Average range width
    average_range_width = df['upper_bound'] - df['lower_bound']
    average_range_width = average_range_width.mean()

    filtered_df = df[df['original'] > 0.5]  # Filter for points > 0.5
    coverage_gt_0_5 = (filtered_df['original'] >= filtered_df['lower_bound']) & (filtered_df['original'] <= filtered_df['upper_bound'])
    coverage_gt_0_5_pct = coverage_gt_0_5.mean() * 100

    return coverage_pct, average_range_width, coverage_gt_0_5_pct

def evaluate_point_predictions(data):
    df = pd.DataFrame(data)  # Ensure DataFrame format

    mse_mean = np.mean((df['original'] - df['point_pred_mean']) ** 2)
    rmse_mean = np.sqrt(mse_mean)

    # MSE and RMSE for median predictions
    mse_median = np.mean((df['original'] - df['point_pred_median']) ** 2)
    rmse_median = np.sqrt(mse_median)

    mse_avg = np.mean((df['original'] - df['lower_upper_avg']) ** 2)
    rmse_avg = np.sqrt(mse_avg)

    # MAE for mean predictions
    mae_mean = np.mean(np.abs(df['original'] - df['point_pred_mean']))

    # MAE for median predictions
    mae_median = np.mean(np.abs(df['original'] - df['point_pred_median']))

    mae_avg = np.mean(np.abs(df['original'] - df['lower_upper_avg']))

    return mse_mean, rmse_mean, mae_mean,  mse_median, rmse_median , mae_median, mse_avg, rmse_avg, mae_avg

def evaluate_range_predictions_new(data):
    df = pd.DataFrame(data)  # Ensure DataFrame format

    # Coverage: Proportion of original values within bounds
    coverage = (df['original'] >= df['lower_bound']) & (df['original'] <= df['upper_bound'])
    coverage_pct = coverage.mean() * 100

    # Average range width for different buckets
    bins = [0, 0.13, 0.25, 0.51, 0.95, 1]  # Define your buckets
    labels = ['0-0.13', '0.13-0.25', '0.25-0.51', '0.51-0.95', '0.95-1']
    df['bucket'] = pd.cut(df['original'], bins=bins, labels=labels, right=False)

    average_range_widths = {}
    for bucket in df['bucket'].unique():
        bucket_df = df[df['bucket'] == bucket]
        average_range_width = (bucket_df['upper_bound'] - bucket_df['lower_bound']).mean()
        average_range_widths[bucket] = average_range_width

    filtered_df = df[df['original'] > 0.5]  # Filter for points > 0.5
    coverage_gt_0_5 = (filtered_df['original'] >= filtered_df['lower_bound']) & (filtered_df['original'] <= filtered_df['upper_bound'])
    coverage_gt_0_5_pct = coverage_gt_0_5.mean() * 100

    return coverage_pct, average_range_widths, coverage_gt_0_5_pct

def evaluate_range_predictions_new_c(data):
    df = pd.DataFrame(data)  # Ensure DataFrame format

    # Coverage: Proportion of original values within bounds
    coverage = (df['original'] >= df['lower_bound']) & (df['original'] <= df['upper_bound'])
    coverage_pct = coverage.mean() * 100

    # Average range width and coverage for different buckets
    bins = [0, 0.13, 0.25, 0.51, 0.95, 1]  # Define your buckets
    labels = ['0-0.13', '0.13-0.25', '0.25-0.51', '0.51-0.95', '0.95-1']
    df['bucket'] = pd.cut(df['original'], bins=bins, labels=labels, right=False)

    average_range_widths = {}
    coverage_percentages = {}
    for bucket in df['bucket'].unique():
        bucket_df = df[df['bucket'] == bucket]
        average_range_width = (bucket_df['upper_bound'] - bucket_df['lower_bound']).mean()
        average_range_widths[bucket] = average_range_width

        # Calculate coverage for the bucket
        bucket_coverage = ((bucket_df['original'] >= bucket_df['lower_bound']) &
                           (bucket_df['original'] <= bucket_df['upper_bound'])).mean() * 100
        coverage_percentages[bucket] = bucket_coverage

    filtered_df = df[df['original'] > 0.5]  # Filter for points > 0.5
    coverage_gt_0_5 = (filtered_df['original'] >= filtered_df['lower_bound']) & (filtered_df['original'] <= filtered_df['upper_bound'])
    coverage_gt_0_5_pct = coverage_gt_0_5.mean() * 100

    return coverage_pct, average_range_widths, coverage_gt_0_5_pct, coverage_percentages


In [ ]:
def print_results_1(data):
  test_coverage_pct, test_average_range_width, test_coverage_pct_gt_0_5_pct = evaluate_range_predictions(data)
  print(f"test Coverage: {test_coverage_pct:.4f}%")
  print(f"test Average Range Width: {test_average_range_width:.4f}")
  print(f"test Coverage > 0.5: {test_coverage_pct_gt_0_5_pct:.4f}%")

def print_results_2(data):
  t_c , t_b_w , t_c_gt_0_5_pct = evaluate_range_predictions_new(data)
  print(f"coverage: {t_c}")
  print(f"average range width: {t_b_w}")
  print(f"coverage > 0.5: {t_c_gt_0_5_pct}")

def print_results_3(data):
  t1,t2,t3,t4 = evaluate_range_predictions_new_c(data)
  print("Coverage percentages per bucket",t4)

def print_point_results(data,datatype):
  test_mse_mean, test_rmse_mean, test_mae_mean, test_mse_median, test_rmse_median, test_mae_median, test_avg_mse, test_avg_rmse, test_avg_mae = evaluate_point_predictions(data)
  print(f"{datatype} mse mean: {test_mse_mean:.4f}")
  print(f"{datatype} rmse mean: {test_rmse_mean:.4f}")
  print(f"{datatype} mae mean: {test_mae_mean:.4f}")
  print(f"{datatype} mse median: {test_mse_median:.4f}")
  print(f"{datatype} rmse median: {test_rmse_median:.4f}")
  print(f"{datatype} mae median: {test_mae_median:.4f}")
  print(f"{datatype} avg mse: {test_avg_mse:.4f}")
  print(f"{datatype} avg rmse: {test_avg_rmse:.4f}")
  print(f"{datatype} avg mae: {test_avg_mae:.4f}")



In [ ]:
def plot_graph(data,title):
    df = pd.DataFrame(data)

    # Select a random subset
    df_plot = df.sample(n=50, random_state=42)

    # Assign sequential indexes (starting from 0)
    df_plot['index'] = range(len(df_plot))

    plt.figure(figsize=(10, 6))  # Adjust figure size as needed

    plt.scatter(df_plot['index'], df_plot['original'], marker='o', s=100, label='Original Values')
    # Scatter plot for median predictions
    plt.scatter(df_plot['index'], df_plot['point_pred_median'], marker='o', s=100, label='Median Predictions', color='red')
    plt.plot(df_plot['index'], df_plot['lower_bound'], color='lightgray', label='Lower Bound')
    plt.plot(df_plot['index'], df_plot['upper_bound'], color='lightgray', label='Upper Bound')

    # Connect upper and lower bounds with lines for clarity
    for i in range(len(df_plot)):
      plt.plot([df_plot['index'].iloc[i], df_plot['index'].iloc[i]],
              [df_plot['lower_bound'].iloc[i], df_plot['upper_bound'].iloc[i]],
              color='gray', linewidth=1)

    plt.xlabel('Index')
    plt.ylabel('Fog Index Values')
    plt.title(title)

    plt.legend()  # Add legend for clarity
    plt.tight_layout()  # Adjust spacing for labels and title
    plt.show()

In [ ]:
# update with the appropriate testing data path
df_winter_test = pd.read_csv('',index_col=0, parse_dates=True)

df_winter_test = df_winter_test.dropna()

X_test = df_winter_test.drop(["fog_index_6h","fog_index_1d","fog_index_3d","fog_index_5d"], axis=1)  # Features
y_test = df_winter_test["fog_index_5d"]  # Target variable

X_test_scaled = scaler.transform(X_test)

X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
with beta_regression_model:
    pm.set_data({
        'avg_air_temp': X_test_scaled_df['avg_air_temp'],
        'avg_dew_point': X_test_scaled_df['avg_dew_point'],
        'avg_relative_humidity': X_test_scaled_df['avg_relative_humidity'],
        'avg_pressure': X_test_scaled_df['avg_pressure'],
        'avg_visibility': X_test_scaled_df['avg_visibility'],
        'energy_loss': X_test_scaled_df['energy_loss'],
        'fog_duration': X_test_scaled_df['fog_duration'],
        'fog_month': X_test_scaled_df['fog_month'],
        'fog_index': y_test
    })

    idata_test = pm.sample_posterior_predictive(
                trace,
                var_names=["mu","fog_index_var"],
                return_inferencedata=True,
                predictions=True,

                random_seed=42  # Or any seed
     )

In [ ]:
idata_test

In [ ]:
print(idata_test)

In [ ]:
print(idata_test.predictions)

In [ ]:
predictions_for_test_point = idata_test.predictions['fog_index_var']  # Sample some draws

# HDI Calculation
hdi = pm.hdi(predictions_for_test_point, hdi_prob=0.95)
print(hdi)

hdi_90 = pm.hdi(predictions_for_test_point, hdi_prob=0.9)
hdi_99 = pm.hdi(predictions_for_test_point, hdi_prob=0.99)

In [ ]:
test_point_predictions_mean = idata_test.predictions['fog_index_var'].mean(dim=['chain', 'draw']).values
test_point_predictions_median = idata_test.predictions['fog_index_var'].median(dim=['chain', 'draw']).values

In [ ]:
test_hdi_95_cal_data = calculate_intervals(hdi,y_test,test_point_predictions_mean,test_point_predictions_median)
test_hdi_90_cal_data = calculate_intervals(hdi_90,y_test,test_point_predictions_mean,test_point_predictions_median)
test_hdi_99_cal_data = calculate_intervals(hdi_99,y_test,test_point_predictions_mean,test_point_predictions_median)

In [ ]:
print_results_1(test_hdi_95_cal_data)

In [ ]:
print_results_1(test_hdi_90_cal_data)

In [ ]:
print_results_1(test_hdi_99_cal_data)

In [ ]:
print_results_2(test_hdi_95_cal_data)

In [ ]:
print_results_2(test_hdi_90_cal_data)

In [ ]:
print_results_2(test_hdi_99_cal_data)

In [ ]:
print_results_3(test_hdi_95_cal_data)

In [ ]:
print_results_3(test_hdi_90_cal_data)

In [ ]:
print_results_3(test_hdi_99_cal_data)

In [ ]:
print_point_results(test_hdi_95_cal_data,"test")

In [ ]:
plot_graph(test_hdi_90_cal_data,"Test Data Predictions - Beta Regression (5 days)")